In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-15 13:46:41,565 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'SCS_Alerts_NOV_new.csv')



In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=3077198555888) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=3077198556320) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=3077198556464) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=3077198553584) was called 1 times
Running: Function "load_module_lookup" (id=3077198554736) was called 1 times
duplicated location faults removed - max duration kept


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [9]:
fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Grey','Both']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'count')

Running: Function "faults_aggregate" (id=3077198555744) was called 1 times


In [10]:
av_sel,at_sel = feat.av_at_select(av, at, remove_high_AT = False,availability_select_options = None)

av_agg = feat.aggregate_availability(av_sel, agg_level = None)
at_agg = feat.aggregate_totes(at_sel, agg_level = None)

#at_agg = at_agg[at_agg['TOTES']>100]

Running: Function "aggregate_availability" (id=3077198725328) was called 1 times


In [11]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = None)

In [12]:
X,y = md.gen_feat_var(df,target = 'Grey Tote Loss')
y=1-y

In [13]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'thres': -np.inf,
                    'max_feat': None}

X_sel_rf = md.select_features(X, y, RF_mdl, selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options)

X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options)

                                                                                                           #X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X_sel_lm, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X_sel_rf, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X_sel_XGB, y)

#md.stats_model(Linear_mdl,X_test,y_test)

Feature Importance Ranking: 
 
    Importance      Feature
0    0.481828        TOTES
1    0.049950       SCSM08
2    0.040136  C1202ACH055
3    0.030181  C1402STA061
4    0.026208  C1104STA197
5    0.022552       SCSM19
6    0.019770       SCSM03
7    0.018654  C1401ACH001
8    0.014409       SCSM13
9    0.013916  C1404STA164

Regression Metrics: 
 
           RF Metrics
MAE         0.028453
MSE         0.002421
RMSE        0.049200
MAPE%       3.660000
ACC%       96.340000
OOB         0.380665
R2_Train    0.918019
R2_Pred     0.201201 

Feature Importance Ranking: 
 
    Importance      Feature
0    0.282085  C1102STA071
1    0.079272        TOTES
2    0.068908  C1202ACH055
3    0.060474  C1104STA173
4    0.053743  C1104STA197
5    0.045704  C0701STA002
6    0.033792  C1102STA056
7    0.031499  C0802RDC094
8    0.029299  C1102STA097
9    0.028080  C1402STA061

Regression Metrics: 
 
          XGB Metrics
MAE        0.0254049
MSE       0.00148208
RMSE       0.0384978
MAPE%           3

### Add LM - metrics

In [14]:
sc.lm_coefficients(Linear_mdl, X, y)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,Feature,Coefficients,Standard Errors,t values,Probabilities
0,Constant,0.9284,NaN,NaN,NaN
1,C05,0.0000,NaN,NaN,NaN
2,C0501ACH011,-0.0006,NaN,NaN,NaN
3,C0501ACH018,-0.0010,NaN,NaN,NaN
4,C0501STA002,-0.0084,NaN,NaN,NaN
...,...,...,...,...,...
627,SCSM17,0.0009,NaN,NaN,NaN
628,SCSM18,-0.0005,NaN,NaN,NaN
629,SCSM19,-0.0048,NaN,NaN,NaN
630,SCSM20,0.0005,NaN,NaN,NaN


In [15]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1698: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1698: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1620: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1446: RuntimeWarning: invalid value encountered in multiply
  cov_p = self.normalized_cov

                            OLS Regression Results                            
Dep. Variable:         Grey Tote Loss   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 15 Dec 2020   Prob (F-statistic):                nan
Time:                        13:47:11   Log-Likelihood:                 2694.9
No. Observations:                  83   AIC:                            -5224.
Df Residuals:                       0   BIC:                            -5023.
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0678        inf          0      

In [16]:
df_greyhigh_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_ID.sum().sort_values(ascending=False).head(20)

SCSM18         1458.0
C09            1389.0
C12            1380.0
SCSM12         1339.0
C14            1167.0
C08            1139.0
C13             915.0
SCSM03          860.0
SCSM15          716.0
C11             671.0
C07             670.0
C06             623.0
SCSM04          615.0
SCSM07          607.0
SCSM11          599.0
SCSM01          593.0
SCSM20          535.0
C1404STA164     494.0
SCSM13          472.0
C05             459.0
dtype: float64

In [17]:
df_grey_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_ID.sum().sort_values(ascending=False).head(20)

SCSM18         1458.0
C09            1389.0
C12            1380.0
SCSM12         1339.0
C14            1167.0
C08            1139.0
C13             915.0
SCSM03          860.0
SCSM15          716.0
C11             671.0
C07             670.0
C06             623.0
SCSM04          615.0
SCSM07          607.0
SCSM11          599.0
SCSM01          593.0
SCSM20          535.0
C1404STA164     494.0
SCSM13          472.0
C05             459.0
dtype: float64

In [18]:
df_greyhigh_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_Asset.sum().sort_values(ascending=False).head(20)

SCSM18         1458.0
C09            1389.0
C12            1380.0
SCSM12         1339.0
C14            1167.0
C08            1139.0
C13             915.0
SCSM03          860.0
SCSM15          716.0
C11             671.0
C07             670.0
C06             623.0
SCSM04          615.0
SCSM07          607.0
SCSM11          599.0
SCSM01          593.0
SCSM20          535.0
C1404STA164     494.0
SCSM13          472.0
C05             459.0
dtype: float64

In [19]:
df_grey_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_Asset.sum().sort_values(ascending=False).head(20)

SCSM18         1458.0
C09            1389.0
C12            1380.0
SCSM12         1339.0
C14            1167.0
C08            1139.0
C13             915.0
SCSM03          860.0
SCSM15          716.0
C11             671.0
C07             670.0
C06             623.0
SCSM04          615.0
SCSM07          607.0
SCSM11          599.0
SCSM01          593.0
SCSM20          535.0
C1404STA164     494.0
SCSM13          472.0
C05             459.0
dtype: float64

In [20]:
X

,C05,C0501ACH011,C0501ACH018,C0501STA002,C0501STA012,C0501STA015,C0501STA020,C0501STA217,C0501STA219,C0502ACH079,...,SCSM11,SCSM12,SCSM13,SCSM14,SCSM15,SCSM17,SCSM18,SCSM19,SCSM20,TOTES
27,2,0,0,0,0,0,0,0,0,0,...,4,4,4,0,5,0,4,1,0,60.557407
28,1,0,0,0,0,0,0,0,0,0,...,1,13,1,0,5,0,10,0,0,63.171296
29,4,0,0,0,0,0,0,0,0,0,...,1,21,2,0,0,0,24,0,0,60.872222
30,8,0,0,0,0,0,0,0,0,0,...,15,2,6,4,0,1,2,0,1,66.359259
31,3,0,0,0,0,0,0,0,0,0,...,7,14,5,0,4,0,3,1,6,66.781481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,3,0,0,0,0,0,0,0,0,0,...,1,2,0,0,11,5,7,2,1,32.274328
142,1,0,0,0,0,0,0,0,0,0,...,5,4,0,1,7,0,18,0,3,40.757407
143,6,0,0,0,0,0,0,0,0,0,...,9,15,1,0,5,0,28,5,6,47.532407
144,3,0,0,0,0,0,0,0,0,0,...,4,15,1,2,8,0,18,1,17,45.150000
